# Preparação dos Dados

Os objetivos deste notebook são:
- Importar os datasets;
- Realizar breve análise descritiva para ganhar domínio e entendimento dos dados.
- Realizar análise de valores faltantes e zerados, e corrigir erros.
- Criar banco de dados, criar schemas e importar os dados.
- Unir os datasets em um único dataframe para realizar os próximos passos.

## Importações de Bibliotecas

In [1]:
import os
import kagglehub

import pandas as pd

from utils import descriptive_analysis as description

import sqlite3

from typing import Dict, List

/home/rnglinux/.pyenv/versions/triggo-ai-challenge/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Funções Utilizadas

In [2]:
def __interpolate_date(start: pd.Series, end: pd.Series,
                      fraction: float) -> pd.Series:
    delta = (end - start)
    
    return start + (delta * fraction)


def __replace_na_values_when_approved_and_carrier_date_var_are(df: pd.DataFrame) -> pd.DataFrame:
    mask_a = (df['order_delivered_customer_date'].notna() &
              df['order_approved_at'].isna() &
              df['order_delivered_carrier_date'].isna())
    
    if mask_a.any():
        df.loc[mask_a, 'order_approved_at'] = __interpolate_date(
            df.loc[mask_a, 'order_purchase_timestamp'],
            df.loc[mask_a, 'order_delivered_customer_date'],
            1/3
        )
        df.loc[mask_a, 'order_delivered_carrier_date'] = __interpolate_date(
            df.loc[mask_a, 'order_approved_at'],
            df.loc[mask_a, 'order_delivered_customer_date'],
            1/2
        )

    return df


def __replace_na_values_when_approved_date_var_is(df: pd.DataFrame) -> pd.DataFrame:
    mask_b = df['order_delivered_customer_date'].notna() & df['order_approved_at'].isna()

    if mask_b.any():
        df.loc[mask_b, 'order_approved_at'] = __interpolate_date(
            df.loc[mask_b, 'order_purchase_timestamp'],
            df.loc[mask_b, 'order_delivered_carrier_date'],
            1/2
        )

    return df


def __replace_na_values_when_carrier_date_var_is(df: pd.DataFrame) -> pd.DataFrame:
    mask_c = df['order_delivered_customer_date'].notna() & df['order_delivered_carrier_date'].isna()

    if mask_c.any():
        df.loc[mask_c, 'order_delivered_carrier_date'] = __interpolate_date(
            df.loc[mask_c, 'order_approved_at'],
            df.loc[mask_c, 'order_delivered_customer_date'],
            1/2
        )

    return df


def __replace_delivered_to_approved(df: pd.DataFrame, delivered_mask) -> pd.DataFrame:
    mask_d = df['order_delivered_customer_date'].isna() & df['order_delivered_carrier_date'].isna()
    if mask_d.any():
        df.loc[delivered_mask & mask_d, 'order_status'] = 'approved'

    return df


def __replace_delivered_to_shipped(df: pd.DataFrame, delivered_mask) -> pd.DataFrame:
    mask_e = df['order_delivered_customer_date'].isna() & df['order_delivered_carrier_date'].notna()
    if mask_e.any():
        df.loc[delivered_mask & mask_e, 'order_status'] = 'shipped'

    return df


def treat_delivered_anomalies(orders: pd.DataFrame) -> pd.DataFrame:
    """
    Corrige valores faltantes nos pedidos cujo order_status == 'delivered'
    seguindo a estratégia combinada:

    - Imputa datas faltantes (‘order_approved_at’, ‘order_delivered_carrier_date’)
      quando ‘order_delivered_customer_date’ está presente.
    - Converte status para 'approved' quando apenas 'order_approved_at' está preenchido.
    - Converte status para 'shipped' quando apenas ‘order_delivered_customer_date’ está ausente.

    Retorna:
        df_corrigido
    """
    na_values_orders = orders[orders.isna().any(axis=1)]
    delivered_mask = na_values_orders['order_status'] == 'delivered'
    df = na_values_orders.loc[delivered_mask]

    date_cols = [
        'order_purchase_timestamp',
        'order_approved_at',
        'order_delivered_carrier_date',
        'order_delivered_customer_date',
        'order_estimated_delivery_date'
    ]

    #df = __change_df_cols_to_datetime(dataframe= df, date_cols= date_cols)

    # --- Etapa 1 - 'order_delivered_customer_date' preenchidos
    # A: 'order_approved_at' & 'order_delivered_carrier_date' faltantes
    df = __replace_na_values_when_approved_and_carrier_date_var_are(df)

    # B: 'order_approved_at' ausentes
    df = __replace_na_values_when_approved_date_var_is(df)

    # C: 'order_delivered_carrier_date' ausentes
    df = __replace_na_values_when_carrier_date_var_is(df)

    # --- Etapa 2 - 'order_delivered_customer_date' ausentes
    # D. 'order_approved_at' preenchidos e 'order_delivered_carrier_date' ausentes
    df = __replace_delivered_to_approved(df, delivered_mask)
    
    # E. só 'order_delivered_customer_date' ausentes
    df = __replace_delivered_to_shipped(df, delivered_mask)
    

    return df

## Importações dos Datasets

In [3]:
# Download dos Datasets
folder_path = kagglehub.dataset_download("olistbr/brazilian-ecommerce")

In [4]:
# Criação de dicionário com datasets importados com dataframes
def clean_file_name(file_name: str) -> str:
    if file_name.startswith("olist_") and file_name.endswith("_dataset.csv"):
        return file_name.removeprefix("olist_").removesuffix("_dataset.csv")
    elif file_name.endswith(".csv"):
        return file_name.removesuffix(".csv")
    return file_name
    
dfs_dict = {}
dfs_name_list = []

for file_name in os.listdir(folder_path):
    if file_name.endswith(".csv"):
        path_file = os.path.join(folder_path, file_name)
        df = pd.read_csv(path_file)

        file_name = clean_file_name(file_name)
        dfs_dict[file_name] = df

## Análise Descritiva e Manipulação de Dados

### Dimensões dos dataframes

In [5]:
description.print_dataframes_size(dfs_dict)

O dataset customers possui 99441 registros e 5 variáveis.
O dataset order_payments possui 103886 registros e 5 variáveis.
O dataset product_category_name_translation possui 71 registros e 2 variáveis.
O dataset order_reviews possui 99224 registros e 7 variáveis.
O dataset products possui 32951 registros e 9 variáveis.
O dataset order_items possui 112650 registros e 7 variáveis.
O dataset orders possui 99441 registros e 8 variáveis.
O dataset sellers possui 3095 registros e 4 variáveis.
O dataset geolocation possui 1000163 registros e 5 variáveis.


In [6]:
description.print_dataframes_cols(dfs_dict)

[
  {
    "dataframe": "customers",
    "colunas": [
      "customer_id",
      "customer_unique_id",
      "customer_zip_code_prefix",
      "customer_city",
      "customer_state"
    ]
  },
  {
    "dataframe": "order_payments",
    "colunas": [
      "order_id",
      "payment_sequential",
      "payment_type",
      "payment_installments",
      "payment_value"
    ]
  },
  {
    "dataframe": "product_category_name_translation",
    "colunas": [
      "product_category_name",
      "product_category_name_english"
    ]
  },
  {
    "dataframe": "order_reviews",
    "colunas": [
      "review_id",
      "order_id",
      "review_score",
      "review_comment_title",
      "review_comment_message",
      "review_creation_date",
      "review_answer_timestamp"
    ]
  },
  {
    "dataframe": "products",
    "colunas": [
      "product_id",
      "product_category_name",
      "product_name_lenght",
      "product_description_lenght",
      "product_photos_qty",
      "product_weight_

In [7]:
description.print_dataframes_types(dfs_dict)

>>> DATASET: customers
customer_id                 object
customer_unique_id          object
customer_zip_code_prefix     int64
customer_city               object
customer_state              object
dtype: object

>>> DATASET: order_payments
order_id                 object
payment_sequential        int64
payment_type             object
payment_installments      int64
payment_value           float64
dtype: object

>>> DATASET: product_category_name_translation
product_category_name            object
product_category_name_english    object
dtype: object

>>> DATASET: order_reviews
review_id                  object
order_id                   object
review_score                int64
review_comment_title       object
review_comment_message     object
review_creation_date       object
review_answer_timestamp    object
dtype: object

>>> DATASET: products
product_id                     object
product_category_name          object
product_name_lenght           float64
product_description_lenght

### Alterando os tipos das variáveis

In [8]:
date_vars = ['shipping_limit_date', 'order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 
'order_delivered_customer_date', 'order_estimated_delivery_date', 'review_creation_date', 'review_answer_timestamp']

# Transformando os tipos de variáveis de data para datetime.
for df in dfs_dict.values():
    for col in date_vars:
        if col in list(df.columns):
            df[col] = pd.to_datetime(df[col])
            df[col] = df[col].dt.floor('min')

# Transformando todos os tipos 'object' dos dataframes para 'category', com o objetivo de aumentar a eficiência nas manipulações.
for df in dfs_dict.values():
    for col in df.select_dtypes(include= 'object').columns:
        df[col] = df[col].astype('category')

In [9]:
description.print_dataframes_types(dfs_dict)

>>> DATASET: customers
customer_id                 category
customer_unique_id          category
customer_zip_code_prefix       int64
customer_city               category
customer_state              category
dtype: object

>>> DATASET: order_payments
order_id                category
payment_sequential         int64
payment_type            category
payment_installments       int64
payment_value            float64
dtype: object

>>> DATASET: product_category_name_translation
product_category_name            category
product_category_name_english    category
dtype: object

>>> DATASET: order_reviews
review_id                        category
order_id                         category
review_score                        int64
review_comment_title             category
review_comment_message           category
review_creation_date       datetime64[ns]
review_answer_timestamp    datetime64[ns]
dtype: object

>>> DATASET: products
product_id                    category
product_category_name     

### Identificando e tratando valores duplicados

In [10]:
datasets_primary_keys_list = [
  {
    'dataframe': 'customers',
    'primary_key': ['customer_id'],
    'foreign_key': ['customer_zip_code_prefix']
  },
  {
    'dataframe': 'order_payments',
    'primary_key': ['order_id', 'payment_sequential'],
    'foreign_key': ['order_id']
  },
  {
    'dataframe': 'product_category_name_translation',
    'primary_key': ['product_category_name'],
    'foreign_key': []
  },
  {
    'dataframe': 'order_reviews',
    'primary_key': ['review_id'],
    'foreign_key': ['order_id']
  },
  {
    'dataframe': 'products',
    'primary_key': ['product_id'],
    'foreign_key': ['product_category_name']
  },
  {
    'dataframe': 'order_items',
    'primary_key': ['order_id', 'order_item_id'], 
    'foreign_key': ['order_id', 'product_id', 'seller_id']
  },
  {
    'dataframe': 'orders',
    'primary_key': ['order_id'],
    'foreign_key': ['customer_id']
  },
  {
    'dataframe': 'sellers',
    'primary_key': ['seller_id'],
    'foreign_key': ['seller_zip_code_prefix']
  },
  {
    'dataframe': 'geolocation',
    'primary_key': ['geolocation_zip_code_prefix'],
    'foreign_key': []
  }
]


In [11]:
description.check_duplicate_values_in_dataframes(dfs_dict, datasets_primary_keys_list)

O dataframe customers não possui valores duplicados.
O dataframe order_payments não possui valores duplicados.
O dataframe product_category_name_translation não possui valores duplicados.
O dataframe order_reviews possui 814 valores duplicados.
O dataframe products não possui valores duplicados.
O dataframe order_items não possui valores duplicados.
O dataframe orders não possui valores duplicados.
O dataframe sellers não possui valores duplicados.
O dataframe geolocation possui 981148 valores duplicados.


> De todos os datasets analisados, foram identificados valores duplicados em <i>geolocation</i> e <i>order reviews</i>. 

**Vamos analisar!**

#### Reviews

In [12]:
reviews = dfs_dict['order_reviews'].copy()

duplicate_review = reviews[reviews.duplicated(subset= ['review_id', 'review_score', 'review_comment_title', 'review_comment_message',
                                                       'review_creation_date', 'review_answer_timestamp'], keep= False)].sort_values('review_id')
agg_review = duplicate_review.groupby('review_id', observed= True)['order_id'].nunique()
agg_review.value_counts()

order_id
2    764
3     25
Name: count, dtype: int64

> Todas as informações das colunas do dataset estão duplicados nos registro identificados como duplicados por 'review_id', menos a coluna 'order_id'! Para cada registro duplicado, a 'order_id' é diferente. Ter dois 'order_id' para o mesmo 'review_id' pode ser um forte indício de erro, pois um review deve estar associado 1:1 com uma order. Estranho... 

- 764 registros foram duplicados 2 vezes, com 'order_id' diferentes.
- 25 registros foram duplicados 3 vezes, com 'order_id' diferentes.

**Vamos analisar os pedidos que possum um mesmo review!**

In [13]:
duplicate_review[0:2]

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
46678,00130cbe1f9d422698c812ed8ded1919,dfcdfc43867d1c1381bfaf62d6b9c195,1,NaN,"O cartucho ""original HP"" 60XL não é reconhecid...",2018-03-07,2018-03-20 18:08:00
29841,00130cbe1f9d422698c812ed8ded1919,04a28263e085d399c97ae49e0b477efa,1,NaN,"O cartucho ""original HP"" 60XL não é reconhecid...",2018-03-07,2018-03-20 18:08:00


In [14]:
orders = dfs_dict['orders'].copy()

orders.loc[orders['order_id'] == 'dfcdfc43867d1c1381bfaf62d6b9c195']

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
74048,dfcdfc43867d1c1381bfaf62d6b9c195,a7026133ddbd2e86c83ecd4dfa4dbe01,delivered,2018-02-02 18:01:00,2018-02-02 18:31:00,2018-02-05 23:46:00,2018-02-26 18:18:00,2018-03-09


In [15]:
orders.loc[orders['order_id'] == '04a28263e085d399c97ae49e0b477efa']

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
28241,04a28263e085d399c97ae49e0b477efa,fef2e5e63da9f3e1dd89e8e319468657,delivered,2018-02-02 18:01:00,2018-02-02 18:31:00,2018-02-21 02:38:00,2018-03-11 11:32:00,2018-03-05


>  Os pedidos possuem 'customer_id' únicos associados (esperado), mas foram realizados quase que no mesmo instante (2 segundos de diferença)... Será que são pedidos feitos pelo mesmo cliente??? 

**Vamos analisar o <i>'customer_unique_id'</i>!**

In [16]:
customers = dfs_dict['customers'].copy()

customers.loc[customers['customer_id'] == 'd39c97549a412a82dacf30fe5f144cb0']

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
5281,d39c97549a412a82dacf30fe5f144cb0,91ffaf5eb7f4bd6a48b07f3546fb7a99,65415,coroata,MA


In [17]:
customers.loc[customers['customer_id'] == '3bb15be93abf3c2a684d163ba6faaa5b']

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
13902,3bb15be93abf3c2a684d163ba6faaa5b,91ffaf5eb7f4bd6a48b07f3546fb7a99,65415,coroata,MA


> As compras possuem o mesmo comprador!

**Comportamento**: Um review de um pedido é duplicado para um outro pedido de um mesmo cliente, se o cliente fizer dois ou mais pedidos em um mesmo instante.

**Obs.**: O comportamento acima é sólido e evidencia claramente um erro que não deveria acontecer.

Como nem todo pedido há uma avaliação, irei excluir todos os registros duplicados que possuem duplicação, pois não se poderá definir qual pedido a avaliação pertence. 

In [18]:
reviews.drop_duplicates(subset= 'review_id', keep= 'first', inplace= True)

dfs_dict['order_reviews'] = reviews

#### Geolocation

O dataset <i>geolocation</i> representa pontos de localização associados nomes de cidades e estados. A chave primária desta dataset é o 'zip code', que não dever haver duplicações. Logo como solução, também irei excluir os registros duplicados.

In [19]:
geolocation = dfs_dict['geolocation'].copy()

geolocation.drop_duplicates(subset= 'geolocation_zip_code_prefix', keep= 'first', inplace= True)

dfs_dict['geolocation'] = geolocation

In [20]:
description.check_duplicate_values_in_dataframes(dfs_dict, datasets_primary_keys_list)

O dataframe customers não possui valores duplicados.
O dataframe order_payments não possui valores duplicados.
O dataframe product_category_name_translation não possui valores duplicados.
O dataframe order_reviews não possui valores duplicados.
O dataframe products não possui valores duplicados.
O dataframe order_items não possui valores duplicados.
O dataframe orders não possui valores duplicados.
O dataframe sellers não possui valores duplicados.
O dataframe geolocation não possui valores duplicados.


> Todos os dados duplicados de <i>geolocation</i> e <i>order reviews</i> foram removidos, mantendo apenas as primeiras ocorrências.

### Identificando e tratando valores faltantes

In [21]:
description.check_missing_values_in_dataframes(dfs_dict)

O dataset customers não possui valores faltantes.
O dataset order_payments não possui valores faltantes.
O dataset product_category_name_translation não possui valores faltantes.
O dataset order_reviews possui valores faltantes:
    A 'review_comment_title' possui 86891 registros faltantes (88.29%)
    A 'review_comment_message' possui 57742 registros faltantes (58.67%)

O dataset products possui valores faltantes:
    A 'product_category_name' possui 610 registros faltantes (1.85%)
    A 'product_name_lenght' possui 610 registros faltantes (1.85%)
    A 'product_description_lenght' possui 610 registros faltantes (1.85%)
    A 'product_photos_qty' possui 610 registros faltantes (1.85%)
    A 'product_weight_g' possui 2 registros faltantes (0.01%)
    A 'product_length_cm' possui 2 registros faltantes (0.01%)
    A 'product_height_cm' possui 2 registros faltantes (0.01%)
    A 'product_width_cm' possui 2 registros faltantes (0.01%)

O dataset order_items não possui valores faltantes.
O 

> Foi identificado que os datasets <i>order reviews, products e orders</i> possuem dados faltantes em colunas específicas.

**Vamos interpretar!**
- O dataset order <i>reviews</i> é o dataset responsável pelas avaliações dos pedidos do cliente. É normal que um cliente não forneça sua avaliação escrita. É o que confirmam os dados faltantes nos campos de título e comentário sobre a experiência de compra. Logo, os dados faltantes para esse dataset não precisam de tratamento.
- O dados faltantes de <i>products</i>, aparentam estar divididos em dois grupos:
    1. 2 registros faltantes em variáveis de dimensão física do produto.
    2. 610 registros faltantes em variávies de identificação do produto.
    - Vamos analisar esse dois grupos e checar o que pode ser feito.

- Os dados faltantes de <i>orders</i> estão possivelmente ligados ao status do pedido. Se o pedido não for aprovado, é normal que não tenha data de aprovação. A lógica segue para as outras variáveis de data faltantes. Irei avaliar a relação destes dados faltantes com a variável 'order_status'.

#### Valores faltantes no dataset de Produtos

In [22]:
# 1. Dimensão Física do Produto
products = dfs_dict['products'].copy()

products.loc[products['product_weight_g'].isna()]

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
8578,09ff539a621711667c43eba6a3bd8466,bebes,60.0,865.0,3.0,NaN,NaN,NaN,NaN
18851,5eb564652db742ff8f28759cd8d2652a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Abordagens
1. Os dados faltantes do registro de id "09ff539a621711667c43eba6a3bd8466", podem ser substituídos por métricas estatísticas de cada variável filtrada pela categoria do produto.
1. Todo o registro de id "5eb564652db742ff8f28759cd8d2652a" está ausente, logo esse registro não tem valor algum para a base de dados. Será excluído.

In [23]:
# 1 - Substituição dos dados faltantes pela mediana
na_cols = ['product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']
na_vars_median_list = products.loc[products['product_category_name'] == "bebes", na_cols].median().to_list()

products.loc[products['product_id'] == "09ff539a621711667c43eba6a3bd8466", na_cols] = na_vars_median_list

# 2 - Exclusão do registro faltante
products.drop(products[products['product_id'] == '5eb564652db742ff8f28759cd8d2652a'].index, inplace=True)

In [24]:
products.loc[products['product_weight_g'].isna()]

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm


In [25]:
# 2. Identificação do Produto
products.loc[products['product_category_name'].isna()]

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
105,a41e356c76fab66334f36de622ecbd3a,NaN,NaN,NaN,NaN,650.0,17.0,14.0,12.0
128,d8dee61c2034d6d075997acef1870e9b,NaN,NaN,NaN,NaN,300.0,16.0,7.0,20.0
145,56139431d72cd51f19eb9f7dae4d1617,NaN,NaN,NaN,NaN,200.0,20.0,20.0,20.0
154,46b48281eb6d663ced748f324108c733,NaN,NaN,NaN,NaN,18500.0,41.0,30.0,41.0
197,5fb61f482620cb672f5e586bb132eae9,NaN,NaN,NaN,NaN,300.0,35.0,7.0,12.0
...,...,...,...,...,...,...,...,...,...
32515,b0a0c5dd78e644373b199380612c350a,NaN,NaN,NaN,NaN,1800.0,30.0,20.0,70.0
32589,10dbe0fbaa2c505123c17fdc34a63c56,NaN,NaN,NaN,NaN,800.0,30.0,10.0,23.0
32616,bd2ada37b58ae94cc838b9c0569fecd8,NaN,NaN,NaN,NaN,200.0,21.0,8.0,16.0
32772,fa51e914046aab32764c41356b9d4ea4,NaN,NaN,NaN,NaN,1300.0,45.0,16.0,45.0


In [26]:
# Variável Numérica ('product_name_lenght', 'product_description_lenght', 'product_photos_qty')
na_numeric_cols = ['product_name_lenght', 'product_description_lenght', 'product_photos_qty']
na_vars_median_list = products[na_numeric_cols].median().to_list()

products.loc[products['product_category_name'].isna(), na_numeric_cols] = na_vars_median_list

# Variável Categórica ('product_category_name')
na_categoric_col = ['product_category_name']
na_vars_mode_list = products[na_categoric_col].mode()['product_category_name'][0]

products.loc[products['product_category_name'].isna(), na_categoric_col] = na_vars_mode_list

In [27]:
dfs_dict['products'] = products

products.loc[products['product_category_name'].isna()]

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm


#### Valores faltantes no dataset de Pedidos

In [28]:
na_values_orders = orders[orders.isna().any(axis=1)]

na_values_orders.groupby('order_status')['order_id'].count()

/tmp/ipykernel_16023/2679850568.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  na_values_orders.groupby('order_status')['order_id'].count()


order_status
approved          2
canceled        619
created           5
delivered        23
invoiced        314
processing      301
shipped        1107
unavailable     609
Name: order_id, dtype: int64

In [29]:
def print_na_values_in_vars_by_order_status() -> None:
    status_list = list(orders['order_status'].unique())

    for status in status_list:
        na_values_df= orders.loc[orders['order_status'] == status]
        print(f'>>> STATUS: {status}')
        print(na_values_df.isna().sum())
        print()

print_na_values_in_vars_by_order_status()


>>> STATUS: delivered
order_id                          0
customer_id                       0
order_status                      0
order_purchase_timestamp          0
order_approved_at                14
order_delivered_carrier_date      2
order_delivered_customer_date     8
order_estimated_delivery_date     0
dtype: int64

>>> STATUS: invoiced
order_id                           0
customer_id                        0
order_status                       0
order_purchase_timestamp           0
order_approved_at                  0
order_delivered_carrier_date     314
order_delivered_customer_date    314
order_estimated_delivery_date      0
dtype: int64

>>> STATUS: shipped
order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                   0
order_delivered_carrier_date        0
order_delivered_customer_date    1107
order_estimated_delivery_date       0
dtype: int64

>>> STA

Ao analisar o comportamento dos dados faltantes, pude validar e reconhecer várias regras de negócio esperadas aplicadas no dataset. Essas regras definem os valores das variáveis de registro de datas e a variável de status do pedido. São elas:

- Quando um pedido for criado, o campo order_purchase_timestamp deve ser preenchido. order_approved_at, order_delivered_carrier_date e order_delivered_customer_date ficam ausentes.
- Quando um pedido for aprovado, ou estiver em fatura, ou em processamento, ou indisponível, os campos order_purchase_timestamp e order_approved_at devem ser preenchidos. order_delivered_carrier_date e order_delivered_customer_date ficam ausentes. 
- Quando um pedido for enviado, os campos order_purchase_timestamp, order_approved_at e order_delivered_carrier_date devem ser preenchidos. order_delivered_customer_date ficam ausentes.
- Quando um pedido é cancelada, a única variável que deve estar preenchida é order_purchase_timestamp, pois um pedido só pode ser cancelado depois de emitido. order_approved_at, order_delivered_carrier_date e order_delivered_customer_date podem ser ausentes ou não, vai depender do momento em que foi cancelado. 

Todas as regras acima, estão validadas pelo comportamento dos dados, e permaneceram com valores ausentes. Porém, há uma inconsistência no comportamento das variáveis de data quando o pedido é entregue. Por lógica, todas as variáveis de data deveriam estar preenchida, quando o pedido é entregue. É o que não acontece em 23 registros.

#### Solução
Para solucionar as inconsistências das variáveis de data quando 'order_status' é igual a "delivery", vou adotar as seguintes estratégias:
- Etapa 1: Registros com order_status == "delivered" e 'order_delivered_customer_date' preenchido.
    - Caso A: Apenas 'order_approved_at' ausente.
        - Ação: Preencher com data interpolada entre order_purchase_timestamp e order_delivered_customer_date.

    - Caso B: Apenas order_delivered_carrier_date ausente.
        - Ação: Preencher com data interpolada entre order_approved_at e order_delivered_customer_date.

    - Caso C: 'order_approved_at' e 'order_delivered_carrier_date' ausentes.
        Ação:
        - Definir data para 'order_approved_at' com data interpolada entre 'order_purchase_timestamp' e 'order_delivered_customer_date';
        - Definir data para 'order_delivered_carrier_date' com data interpolada entre o 'order_approved_at' estimado e 'order_delivered_customer_date'.

- Etapa 2: Registros com 'order_status' == "delivered" e 'order_delivered_customer_date' ausente.
    - Caso D: 'order_approved_at' preenchido e 'order_delivered_carrier_date' ausentes
        - Ação: Alterar order_status para "approved".
        - Justificativa: para que um pedido possua uma data de aprovação, ele necessariamente em algum momento foi aprovado.

    - Caso E: Apenas 'order_delivered_customer_date' ausente
        - Ação: Alterar order_status para "shipped".
        - Justificativa: O pedido foi enviado (carrier_date presente), mas não chegou ao cliente — está em trânsito.

In [30]:
new_df = treat_delivered_anomalies(orders = orders)
new_df

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
3002,2d1e2d5bf4dc7227b3bfebb81328c15f,ec05a6d8558c6455f0cbbd8a420ad34f,shipped,2017-11-28 17:44:00,2017-11-28 17:56:00,2017-11-30 18:12:00,NaT,2017-12-18
5323,e04abd8149ef81b95221e88f6ed9ab6a,2127dc6603ac33544953ef05ec155771,delivered,2017-02-18 14:40:00,2017-02-21 01:22:00,2017-02-23 12:04:00,2017-03-01 13:25:00,2017-03-17
16567,8a9adc69528e1001fc68dd0aaebbb54a,4c1ccc74e00993733742a3c786dc3c1f,delivered,2017-02-18 12:45:00,2017-02-20 22:53:00,2017-02-23 09:01:00,2017-03-02 10:05:00,2017-03-21
19031,7013bcfc1c97fe719a7b5e05e61c12db,2941af76d38100e0f8740a374f1a5dc3,delivered,2017-02-18 13:29:00,2017-02-20 14:57:00,2017-02-22 16:25:00,2017-03-01 08:07:00,2017-03-17
20618,f5dd62b788049ad9fc0526e3ad11a097,5e89028e024b381dc84a13a3570decb4,shipped,2018-06-20 06:58:00,2018-06-20 07:19:00,2018-06-25 08:05:00,NaT,2018-07-16
22663,5cf925b116421afa85ee25e99b4c34fb,29c35fc91fc13fb5073c8f30505d860d,delivered,2017-02-18 16:48:00,2017-02-20 14:05:30,2017-02-22 11:23:00,2017-03-09 07:28:00,2017-03-31
23156,12a95a3c06dbaec84bcfb0e2da5d228a,1e101e0daffaddce8159d25a8e53f2b2,delivered,2017-02-17 13:05:00,2017-02-20 00:14:00,2017-02-22 11:23:00,2017-03-02 11:09:00,2017-03-20
26800,c1d4211b3dae76144deccd6c74144a88,684cb238dc5b5d6366244e0e0776b450,delivered,2017-01-19 12:48:00,2017-01-22 13:52:00,2017-01-25 14:56:00,2017-01-30 18:16:00,2017-03-01
38290,d69e5d356402adc8cf17e08b5033acfb,68d081753ad4fe22fc4d410a9eb1ca01,delivered,2017-02-19 01:28:00,2017-02-21 02:19:30,2017-02-23 03:11:00,2017-03-02 03:41:00,2017-03-27
39334,d77031d6a3c8a52f019764e68f211c69,0bf35cac6cc7327065da879e2d90fae8,delivered,2017-02-18 11:04:00,2017-02-20 21:13:30,2017-02-23 07:23:00,2017-03-02 16:15:00,2017-03-22


In [31]:
#Merge dos dados corrigidos com o dataset orders
orders.set_index('order_id', inplace=True)
new_df.set_index('order_id', inplace=True)

orders.update(new_df)  

orders.reset_index(inplace=True)

In [32]:
# Checando se o tratamento funcionou:
na_values_orders = orders[orders.isna().any(axis=1)]
delivered_mask = na_values_orders['order_status'] == 'delivered'
na_values_orders.loc[delivered_mask]

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date


In [33]:
na_values_orders.groupby('order_status', observed= False)['order_id'].count()

order_status
approved          3
canceled        619
created           5
delivered         0
invoiced        314
processing      301
shipped        1114
unavailable     609
Name: order_id, dtype: int64

In [34]:
dfs_dict['orders'] = orders

description.check_missing_values_in_dataframes(dfs_dict)

O dataset customers não possui valores faltantes.
O dataset order_payments não possui valores faltantes.
O dataset product_category_name_translation não possui valores faltantes.
O dataset order_reviews possui valores faltantes:
    A 'review_comment_title' possui 86891 registros faltantes (88.29%)
    A 'review_comment_message' possui 57742 registros faltantes (58.67%)

O dataset products não possui valores faltantes.
O dataset order_items não possui valores faltantes.
O dataset orders possui valores faltantes:
    A 'order_approved_at' possui 146 registros faltantes (0.15%)
    A 'order_delivered_carrier_date' possui 1782 registros faltantes (1.79%)
    A 'order_delivered_customer_date' possui 2965 registros faltantes (2.98%)

O dataset sellers não possui valores faltantes.
O dataset geolocation não possui valores faltantes.


### Identificando e tratando valores zerados

In [35]:
description.check_zero_values_in_dataframes(dfs_dict)

O dataframe customers não possui valores zerados.
O dataframe order_payments possui valores zerados:
    A 'payment_installments' possui 2 registros zerados (0.00%)
    A 'payment_value' possui 9 registros zerados (0.01%)

O dataframe order_reviews não possui valores zerados.
O dataframe products possui valores zerados:
    A 'product_weight_g' possui 4 registros zerados (0.01%)

O dataframe order_items possui valores zerados:
    A 'freight_value' possui 383 registros zerados (0.34%)

O dataframe sellers não possui valores zerados.
O dataframe geolocation não possui valores zerados.


Das variáveis que apresentaram valores zerados, as únicas que não fazem sentido possui são:
- 'payment_installments': número de parcelas;
- 'product_weight_g': peso do produto em grama;

Logo, para essas variáveis aplicarei tratamentos adequados.

In [36]:
# payment_installments
payments = dfs_dict['order_payments'].copy()

payments.loc[(payments['payment_installments'] == 0), 'payment_installments'] = payments['payment_installments'].mode()

In [37]:
dfs_dict['order_payments'] = payments

payments.loc[(payments['payment_installments'] == 0)]

,order_id,payment_sequential,payment_type,payment_installments,payment_value


In [38]:
# product_weight_g
products.loc[products['product_weight_g'] == 0, 'product_weight_g'] = products['product_weight_g'].median()

dfs_dict['products'] = products

In [39]:
description.check_zero_values_in_dataframes(dfs_dict)

O dataframe customers não possui valores zerados.
O dataframe order_payments possui valores zerados:
    A 'payment_value' possui 9 registros zerados (0.01%)

O dataframe order_reviews não possui valores zerados.
O dataframe products não possui valores zerados.
O dataframe order_items possui valores zerados:
    A 'freight_value' possui 383 registros zerados (0.34%)

O dataframe sellers não possui valores zerados.
O dataframe geolocation não possui valores zerados.


## Contrução da Base de Dados e Migração 

In [40]:
orders = dfs_dict['orders']
order_items = dfs_dict['order_items']
payments = dfs_dict['order_payments']
order_reviews = dfs_dict['order_reviews']
products = dfs_dict['products']
customers = dfs_dict['customers']
sellers = dfs_dict['sellers']
geolocation = dfs_dict['geolocation']

In [41]:
db_path = '../data/olist.sqlite'

conn = sqlite3.connect(db_path)
cursor = conn.cursor()

In [42]:
geolocation_schema = """
CREATE TABLE IF NOT EXISTS geolocations (
    geolocation_zip_code_prefix TEXT,
    geolocation_lat REAL,
    geolocation_lng REAL,
    geolocation_city TEXT,
    geolocation_state TEXT,
    PRIMARY KEY (geolocation_zip_code_prefix)
);
"""
customers_schema = """
CREATE TABLE IF NOT EXISTS customers (
    customer_id TEXT,
    customer_unique_id TEXT,
    customer_zip_code_prefix TEXT,
    customer_city TEXT,
    customer_state TEXT,
    PRIMARY KEY (customer_id),
    FOREIGN KEY (customer_zip_code_prefix) REFERENCES geolocations(geolocation_zip_code_prefix)
);
"""
sellers_schema = """
CREATE TABLE IF NOT EXISTS sellers (
    seller_id TEXT,
    seller_zip_code_prefix TEXT,
    seller_city TEXT,
    seller_state TEXT,
    PRIMARY KEY (seller_id),
    FOREIGN KEY (seller_zip_code_prefix) REFERENCES geolocations(geolocation_zip_code_prefix)
);
"""
category_name_translation_schema = """
CREATE TABLE IF NOT EXISTS product_category_name_translation (
    product_category_name TEXT,
    product_category_name_english TEXT,
    PRIMARY KEY (product_category_name)
);
"""
products_schema = """
CREATE TABLE IF NOT EXISTS products (
    product_id TEXT,
    product_category_name TEXT,
    product_name_lenght INTEGER,
    product_description_lenght INTEGER,
    product_photos_qty INTEGER,
    product_weight_g REAL,
    product_length_cm REAL,
    product_height_cm REAL,
    product_width_cm REAL,
    PRIMARY KEY (product_id),
    FOREIGN KEY (product_category_name) REFERENCES product_category_name_translation(product_category_name)
);
"""
orders_schema = """
CREATE TABLE IF NOT EXISTS orders (
    order_id TEXT,
    customer_id TEXT,
    order_status TEXT,
    order_purchase_timestamp DATETIME,
    order_approved_at DATETIME,
    order_delivered_carrier_date DATE,
    order_delivered_customer_date DATE,
    order_estimated_delivery_date DATE,
    PRIMARY KEY (order_id),
    FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
);
"""
order_items_schema = """
CREATE TABLE IF NOT EXISTS order_items (
    order_id TEXT,
    order_item_id INTEGER,
    product_id TEXT,
    seller_id TEXT,
    shipping_limit_date DATE,
    price REAL,
    freight_value REAL,
    PRIMARY KEY (order_id, order_item_id),
    FOREIGN KEY (order_id) REFERENCES orders(order_id),
    FOREIGN KEY (product_id) REFERENCES products(product_id),
    FOREIGN KEY (seller_id) REFERENCES sellers(seller_id)
);
"""
order_reviews_schema = """
CREATE TABLE IF NOT EXISTS order_reviews (
    review_id TEXT,
    order_id TEXT,
    review_score INTEGER,
    review_comment_title TEXT,
    review_comment_message TEXT,
    review_creation_date DATE,
    review_answer_timestamp DATETIME,
    PRIMARY KEY (review_id),
    FOREIGN KEY (order_id) REFERENCES ORDERS(order_id)
);
"""
payments_schema = """
CREATE TABLE IF NOT EXISTS order_payments (
    order_id TEXT,
    payment_sequential INTEGER,
    payment_type TEXT,
    payment_installments INTEGER,
    payment_value REAL,
    PRIMARY KEY (order_id, payment_sequential),
    FOREIGN KEY (order_id) REFERENCES orders(order_id)
);
"""

In [43]:
datasets_schema_list = [geolocation_schema, customers_schema, sellers_schema, category_name_translation_schema, 
                        products_schema, orders_schema, order_items_schema, order_reviews_schema, payments_schema]

def create_database_tables(datasets_schema: List) -> None:
    for schema in datasets_schema:
        cursor.execute(schema)

    return None

create_database_tables(datasets_schema_list)
conn.commit()

In [44]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tabelas = cursor.fetchall()

print(">>> Tabelas disponíveis no banco de dados:")
print([t[0] for t in tabelas])

>>> Tabelas disponíveis no banco de dados:
['geolocations', 'customers', 'sellers', 'product_category_name_translation', 'products', 'orders', 'order_items', 'order_reviews', 'order_payments', 'geolocation']


In [ ]:
def add_datasets_in_db(dataframes: Dict) -> None:
    for key, df in dataframes.items():
        df.to_sql(key, con= conn, if_exists= 'append', index= False)

add_datasets_in_db(dfs_dict)
conn.commit()

In [46]:
# Testa se a tabela 'orders' tem dados
query = "SELECT * FROM orders LIMIT 1"
df_test = pd.read_sql_query(query, conn)

print(">>> Primiero registro do DATASET orders:")
print(df_test)

>>> Primiero registro do DATASET orders:
                           order_id                       customer_id  \
0  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   

  order_status order_purchase_timestamp    order_approved_at  \
0    delivered      2017-10-02 10:56:00  2017-10-02 11:07:00   

  order_delivered_carrier_date order_delivered_customer_date  \
0          2017-10-04 19:55:00           2017-10-10 21:25:00   

  order_estimated_delivery_date  
0           2017-10-18 00:00:00  


In [47]:
conn.close()

## União dos DataFrames

In [48]:
description.print_dataframes_size(dfs_dict)

O dataset customers possui 99441 registros e 5 variáveis.
O dataset order_payments possui 103886 registros e 5 variáveis.
O dataset product_category_name_translation possui 71 registros e 2 variáveis.
O dataset order_reviews possui 98410 registros e 7 variáveis.
O dataset products possui 32950 registros e 9 variáveis.
O dataset order_items possui 112650 registros e 7 variáveis.
O dataset orders possui 99441 registros e 8 variáveis.
O dataset sellers possui 3095 registros e 4 variáveis.
O dataset geolocation possui 19015 registros e 5 variáveis.


In [49]:
df = orders.merge(order_items, on= 'order_id', how= 'left')
df = df.merge(products, on= 'product_id', how= 'left')

In [50]:
df[df.duplicated(['order_id', 'order_item_id'], keep= False)]

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,...,price,freight_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm


In [51]:
sellers = sellers.merge(geolocation[['geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng']], left_on= 'seller_zip_code_prefix', right_on= 'geolocation_zip_code_prefix', how= 'left')
sellers.rename(columns= {'geolocation_lat': 'seller_lat', 'geolocation_lng': 'seller_lng'}, inplace= True)

customers = customers.merge(geolocation[['geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng']], left_on= 'customer_zip_code_prefix', right_on= 'geolocation_zip_code_prefix', how= 'left')
customers.rename(columns= {'geolocation_lat': 'customer_lat', 'geolocation_lng': 'customer_lng'}, inplace= True)

sellers.drop(columns= 'geolocation_zip_code_prefix', inplace= True)
customers.drop(columns= 'geolocation_zip_code_prefix', inplace= True)

In [52]:
df = df.merge(sellers, on= 'seller_id', how= 'left')
df = df.merge(customers, on= 'customer_id', how= 'left')

In [53]:
delay_prediction_df = df.copy()

In [54]:
df = df.merge(payments, on= 'order_id', how= 'left')
df = df.merge(order_reviews, on= 'order_id', how= 'left')

### Checando a união dos dataframes

In [55]:
print(f'O dataframe possui {df.shape[0]} registro e {df.shape[1]} variáveis.')

O dataframe possui 118763 registro e 43 variáveis.


In [56]:
df.isna().sum()

order_id                              0
customer_id                           0
order_status                          0
order_purchase_timestamp              0
order_approved_at                   161
order_delivered_carrier_date       2074
order_delivered_customer_date      3403
order_estimated_delivery_date         0
order_item_id                       830
product_id                          830
seller_id                           830
shipping_limit_date                 830
price                               830
freight_value                       830
product_category_name               849
product_name_lenght                 849
product_description_lenght          849
product_photos_qty                  849
product_weight_g                    849
product_length_cm                   849
product_height_cm                   849
product_width_cm                    849
seller_zip_code_prefix              830
seller_city                         830
seller_state                        830


## Salvamento dos Dados para AED

In [57]:
df.to_csv('../data/clean_general_df.csv.gz', index=False, compression="gzip" )
delay_prediction_df.to_csv('../data/delay_prediction_df.csv.gz', index= False, compression="gzip")